In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import utils
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_mcp_adapters.client import MultiServerMCPClient
from mcp import ClientSession, StdioServerParameters

from pydantic import BaseModel, Field
from typing import List, Optional, Generic, TypeVar
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()
API_KEY = os.environ["GOOGLE_AI_KEY"]
llm_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro-exp-03-25",
    api_key=API_KEY
)
utils.tracing_is_enabled()

True

In [6]:
TKey = TypeVar("TKey")
TValue = TypeVar("TValue")
class KeyValueStep(BaseModel, Generic[TKey, TValue]):
    key: TKey
    value: TValue

    def to_list(self) -> List[TValue]:
        return [self.key, self.value]
    
    def to_dict(self) -> dict:
        return {
            self.key: self.value
        }

class CookingRecipe(BaseModel):
    gradients: List[str] = Field(description="gradients which needed to cook the dish")
    steps: List[KeyValueStep[str, str]] = Field(description="Step by step how to make the dish")

    def to_dict(self) -> dict:
        steps = [step.to_list() for step in self.steps]
        steps = {
            step[0]: step[1] for step in steps
        }
        return {
            "gradients": self.gradients,
            "steps": steps
        }

structured_llm = llm_model.with_structured_output(CookingRecipe)
prompt = """You are a professional chef, your duty is receiving request from user about a dish and answer how to cook that in a format.
You need to return 2 things: 
 - First is required gradients, return as a list with each element is each gradient for the dish
 - Second is step by step how to cook the dish, return as a list, each element is a list of 2 strings, first is step order and second is the content of that step
 for the step order, you need to return like this step_1, step_2, step_3, ... return it as a normal string
Example:
- Gradients: ["flour", "sugar", "egg", "milk", ...]
- Steps: [["step_1", "Mix flour and sugar together"], ["step_2", "Add egg and milk to the mixture"], ...]
The request from user: {user_request}
Return your answer here:
"""
answer: CookingRecipe = structured_llm.invoke(
    prompt.format(
        user_request="pizza"
    )
)
print(answer.gradients)
print(answer.steps)
print(answer.to_dict())

['Pizza dough (store-bought or homemade)', 'Pizza sauce', 'Shredded mozzarella cheese', 'Toppings (e.g., pepperoni, mushrooms, bell peppers, onions)', 'Olive oil', 'Cornmeal (optional, for dusting)', 'Flour (for dusting)']
[KeyValueStep[str, str](key='step_1', value='Preheat your oven to 475°F (245°C). If using a pizza stone or steel, place it in the oven now. Lightly dust a pizza peel or the back of a baking sheet with cornmeal or flour to prevent sticking.'), KeyValueStep[str, str](key='step_2', value='On a lightly floured surface, stretch or roll out the pizza dough to your desired thickness (usually 12-14 inches round). Carefully transfer the dough onto the prepared pizza peel or baking sheet.'), KeyValueStep[str, str](key='step_3', value='Brush the edge of the dough lightly with olive oil (optional, for a crispier crust). Spread a layer of pizza sauce evenly over the dough, leaving about a half-inch border for the crust.'), KeyValueStep[str, str](key='step_4', value='Sprinkle the 

In [25]:
async with MultiServerMCPClient() as client:
    await client.connect_to_server(
        server_name="math",
        transport="sse",
        url="http://localhost:8000/sse",
    )
    await client.connect_to_server(
        server_name="Arxiv",
        transport="sse",
        url="http://localhost:9000/sse",
    )
    await client.connect_to_server(
        server_name="Wikipedia",
        transport="sse",
        url="http://localhost:6000/sse",
    )
    agent = create_react_agent(model=llm_model, tools=client.get_tools())
    user_question = "groundino trex 2 paper"
    response = await agent.ainvoke({"messages": user_question})

for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

groundino trex 2 paper
================================== Ai Message ==================================
Tool Calls:
  get_arxiv_papers (02ca1aa3-f6d2-4dc2-bf39-d865ce401473)
 Call ID: 02ca1aa3-f6d2-4dc2-bf39-d865ce401473
  Args:
    query: groundino trex 2
================================= Tool Message =================================
Name: get_arxiv_papers

["{\"id\": null, \"metadata\": {\"Entry ID\": \"http://arxiv.org/abs/1404.0541v3\", \"Published\": \"2015-05-24\", \"Title\": \"Don't Fall for Tuning Parameters: Tuning-Free Variable Selection in High Dimensions With the TREX\", \"Authors\": \"Johannes Lederer, Christian M\\u00fcller\"}, \"page_content\": \"Lasso is a seminal contribution to high-dimensional statistics, but it hinges\\non a tuning parameter that is difficult to calibrate in practice. A partial\\nremedy for this problem is Square-Root Lasso, because it inherently calibrates\\nto the n